# Hướng khai thác dữ liệu

**1. Phân tích mô tả dữ liệu**
- Tần suất phân bố:
Phân bố của positive_ratio, negative_ratio, comment_length theo từng Rating.
So sánh giữa các bình luận tích cực và tiêu cực.
- Tương quan giữa các features:
Kiểm tra xem các đặc trưng nào có mối liên hệ mạnh với nhau hoặc với Rating.
- Phân bố độ dài bình luận (comment_length):
Xem xét liệu bình luận dài hơn có xu hướng tích cực hơn hay không.

**2. Dự đoán sentiment (Rating)**
- Sử dụng các mô hình phân loại để dự đoán Rating dựa trên các features:
Mô hình tuyến tính: Logistic Regression.
Mô hình phi tuyến: Random Forest, Gradient Boosting (XGBoost, LightGBM).
Deep Learning: Mạng Neural (DNN) đơn giản với Keras/TensorFlow.

**3. Trích xuất thông tin cảm xúc**
- Xây dựng chỉ số cảm xúc:
Tổng hợp các từ tích cực/tiêu cực trong văn bản để đưa ra "score" đánh giá cảm xúc của bình luận.
- Phân cụm sentiment:
Dùng thuật toán phân cụm (clustering) để nhóm các bình luận theo mức độ cảm xúc (ví dụ: K-means, DBSCAN).

**4. Ứng dụng vào bài toán thực tế**
- Đánh giá sản phẩm/dịch vụ:
Xác định các từ khóa hoặc cụm từ tích cực/tiêu cực phổ biến trong các đánh giá.
- Phân tích hành vi người dùng:
Tìm hiểu xem người dùng có xu hướng viết bình luận tích cực/tiêu cực dài hay ngắn.
- Phát hiện cảm xúc bất thường:
Dùng các features như negative_ratio để phát hiện các bình luận tiêu cực bất thường và xử lý nhanh.

**5. Tăng cường dữ liệu**
- Kết hợp thêm từ điển cảm xúc:
Tích hợp các danh sách từ tích cực/tiêu cực từ các nguồn bên ngoài.
Làm giàu features:
Thêm các đặc trưng mới như positive_word_count, punctuation_count hoặc uppercase_ratio.

# Importing

In [ ]:
!pip install underthesea

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import re, string
from collections import Counter
from underthesea import word_tokenize

# Read the Data

In [ ]:
data = pd.read_csv('/kaggle/input/vietnamese-sentiment-analysis-food-reviews/vsa_food_rv_train.csv')

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.dropna(inplace=True)

# EDA

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
data.head()

In [ ]:
data['Rating'].value_counts()

In [ ]:
sns.countplot(x='Rating', data = data)

Clean the Data

In [ ]:
def normalize_text(text):

    #Remove các ký tự kéo dài: vd: đẹppppppp
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)

    # Chuyển thành chữ thường
    text = text.lower()

    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
        "👹": " tệ  ", '👍': '  ngon  ',
        "💎": " ngon ", "💩": " tệ  ","😕": " tệ  ", "😱": " tệ  ",
        "🚫": " tệ  ",  "🤬": " tệ  ","🧡": " ngon ",
        '👎': '  tệ   ', '😣': '  tệ   ','✨': '  ngon  ', '❣': '  ngon  ','☀': '  ngon  ',
        '♥': '  ngon  ', '🤩': '  ngon  ', 'like': '  ngon  ',
        '🖤': '  ngon  ', '🤤': '  ngon  ', ':(': '  tệ   ', '😢': '  tệ   ',
        '❤': '  ngon  ', '😍': '  ngon  ', '😘': '  ngon  ', '😪': '  tệ   ', '😊': '  ngon  ',
        '?': ' ? ', '😁': '  ngon  ', '💖': '  ngon  ', '😟': '  tệ   ', '😭': '  tệ   ',
        '💯': '  ngon  ', '💗': '  ngon  ', '♡': '  ngon  ', '💜': '  ngon  ', '🤗': '  ngon  ',
        '^^': '  ngon  ', '😨': '  tệ   ', '☺': '  ngon  ', '💋': '  ngon  ', '👌': '  ngon  ',
        '😖': '  tệ   ', ':((': '  tệ   ', '😡': '  tệ   ', '😠': '  tệ   ',
        '😒': '  tệ   ', '😏': '  tệ   ', '😝': '  ngon  ', '😄': '  ngon  ',
        '😙': '  ngon  ', '😤': '  tệ   ', '😎': '  ngon  ', '😆': '  ngon  ', '💚': '  ngon  ',
        '✌': '  ngon  ', '💕': '  ngon  ', '😞': '  tệ   ', '😓': '  tệ   ', '️🆗️': '  ngon  ',
        '😉': '  ngon  ', '😂': '  ngon  ', '😋': '  ngon  ',
        '💓': '  ngon  ', '😐': '  tệ   ', ':3': '  ngon  ', '😫': '  tệ   ', '😥': '  tệ   ',
        '😬': ' 😬 ', '😌': '  tệ  ', '💛': '  ngon  ', '🤝': '  ngon  ',
        '😗': '  ngon  ', '🤔': '  tệ   ', '😑': '  tệ   ', '🙏': '  tệ   ',
        '😻': '  ngon  ', '💙': '  ngon  ', '💟': '  ngon  ', '🍹':' ngon ', '😀':' ngon ','😃': ' ngon ',
        '😚': '  ngon  ', '❌': '  tệ   ', '👏': '  ngon  ', ';)': '  ngon  ', '<3': '  ngon  ',
        '🌷': '  ngon  ', '🌸': '  ngon  ', '🌺': '  ngon  ',
        '🌼': '  ngon  ', '🍓': '  ngon  ', '🐅': '  ngon  ', '🐾': '  ngon  ', '👉': '  ngon  ',
        '💐': '  ngon  ', '💞': '  ngon  ', '💥': '  ngon  ', '💪': '  ngon  ',
        '😇': '  ngon  ', '😛': '  ngon  ', '😜': '  ngon  ',
        '☹': '  tệ   ',  '💀': '  tệ   ',
        '😔': '  tệ   ', '😧': '  tệ   ', '😩': '  tệ   ', '😰': '  tệ   ',
        '😵': '  tệ   ', '😶': '  tệ   ', '🙁': '  tệ   ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '   ngon  ', ':)': '  ngon  ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u'  ngon  ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': '  ngon  ','hehe': '  ngon  ','hihi': '  ngon  ', 'haha': '  ngon  ', 'hjhj': '  ngon  ',
        ' lol ': '  tệ   ',' cc ': '  tệ   ','cute': u' dễ thương ','huhu': '  tệ   ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u'  ngon  ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': ' tệ ','fresh': ' tươi ','sad': '  tệ  ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u'  ngon  ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u'  ngon  ',
        ' por ': u'  tệ  ',' poor ': u'  tệ  ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)

    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    # texts = text.split()
    # len_text = len(texts)
    #remove những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

In [ ]:
data['cleaned_comment'] = data['Comment'].apply(normalize_text)

In [ ]:
print(data['cleaned_comment'].isnull().sum())

In [ ]:
print(data['cleaned_comment'].head())

In [ ]:
data['comment_length'] = data['cleaned_comment'].apply(len)
sns.histplot(data['comment_length'], bins=30)
plt.title("Độ dài comment")
plt.show()

In [ ]:
print(data.groupby('Rating')['comment_length'].describe())
sns.boxplot(x='Rating', y='comment_length', data=data)
plt.title("Độ dài comment theo Rating")
plt.show()

- Comment với rating tích cực (1.0) có xu hướng dài hơn so với comment tiêu cực (0.0).
- Độ lệch chuẩn tương đối lớn ở cả hai nhóm cho thấy sự phân tán mạnh mẽ về độ dài comment trong mỗi nhóm rating.
- có thể thấy comment tích cực (Rating 1.0) thường dài hơn comment tiêu cực (Rating 0.0) ở mọi mức phân vị
- một số comment rất dài xuất hiện ở cả hai nhóm rating.

Tạo các feature mới

In [ ]:
def load_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        words = f.read().splitlines()
    return words

# Đọc các từ tích cực và tiêu cực
positive_words = load_words('/kaggle/input/sentiment-dicts/pos.txt')
negative_words = load_words('/kaggle/input/sentiment-dicts/nag.txt')

print("Từ tích cực:", positive_words)
print("Từ tiêu cực:", negative_words)


In [ ]:
def extract_features(text, positive_words, negative_words):
    words = text.split()
    total_words = len(words)
    
    # Đếm số từ tích cực/tiêu cực
    positive_count = sum(1 for word in words if word in positive_words)
    negative_count = sum(1 for word in words if word in negative_words)
    
    # Tính tỷ lệ
    positive_ratio = positive_count / total_words if total_words > 0 else 0
    negative_ratio = negative_count / total_words if total_words > 0 else 0
    return {
        'positive_ratio': positive_ratio,
        'negative_ratio': negative_ratio,
        'word_count': total_words,
    }


In [ ]:
# Tạo cột feature từ hàm extract_features
features = data['cleaned_comment'].apply(
    lambda x: extract_features(x, positive_words, negative_words)
)

# Chuyển dict features thành DataFrame
features_df = pd.DataFrame(list(features))
print("Số dòng trong data:", len(data))
print("Số dòng trong features_df:", len(features_df))
# Gộp features vào tập dữ liệu gốc
data = pd.concat([data, features_df], axis=1)


In [ ]:
data = data.dropna(subset=['cleaned_comment'])


In [ ]:
print("Số lượng null trong cleaned_comment sau khi xử lý:", data['cleaned_comment'].isnull().sum())


In [ ]:
data.head()

Tính Correlation Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Tính ma trận tương quan
correlation_matrix = data[[
    'comment_length',
    'positive_ratio',
    'negative_ratio',
    'word_count',
    'Rating']].corr()

# Vẽ heatmap để trực quan hóa
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()


Các feature tương quan nhau không quá cao nên không cần loại bỏ

Tương quan giữa features và nhãn đầu ra

In [ ]:
# Xem tương quan giữa features và nhãn
target_correlation = correlation_matrix['Rating'].sort_values(ascending=False)
print(target_correlation)

Phân tích hệ số tương quan:
- positive_ratio: Tương quan dương nhẹ (0.074) với Rating. Điều này cho thấy rằng văn bản có tỷ lệ từ tích cực cao hơn có thể liên quan một chút đến đánh giá tích cực hơn, nhưng không mạnh mẽ.
- comment_length: Tương quan dương nhẹ (0.068) với Rating. Văn bản dài hơn có xu hướng mang đánh giá tích cực hơn một chút.
- word_count: Tương quan rất yếu (0.015) với Rating. Số lượng từ dường như không ảnh hưởng lớn đến Rating.
- negative_ratio: Tương quan âm (-0.084) với Rating. Tỷ lệ từ tiêu cực có tương quan ngược với đánh giá, nhưng mối quan hệ vẫn yếu.

Loại bỏ word_count, vì thông tin này đã được phản ánh trong comment_length.

In [ ]:
data = data.drop(columns=['word_count'])

# Prepocess

In [ ]:
!pip install pyvi

In [ ]:
from pyvi.ViTokenizer import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
# Vietnamese stopwords list
STOPWORDS = '/kaggle/input/vietnamese-sentiment-analysis-food-reviews/vietnamese-stopwords-dash.txt'
with open(STOPWORDS, "r") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

In [ ]:
def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)
    return train_sentences

def preprocess(text, tokenized = True, lowercased = True):
    text = ViTokenizer.tokenize(text) if tokenized else text
    text = filter_stop_words(text, stopwords)
    text = text.lower() if lowercased else text
    return text

In [ ]:
data.info()

In [ ]:
data['preprocessed_comment'] = data['cleaned_comment'].apply(preprocess)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
data[['comment_length','positive_ratio','negative_ratio']] = scaler.fit_transform(data[['comment_length','positive_ratio','negative_ratio']])

# Training

In [ ]:
data.info()

In [ ]:
data.dropna(inplace=True)

In [ ]:
X = data[['preprocessed_comment', 'comment_length', 'positive_ratio', 'negative_ratio']]
y = data['Rating']

# (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(X_test)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)

# Chuyển văn bản thành các đặc trưng TF-IDF
X_train_tfidf = tfidf.fit_transform(X_train['preprocessed_comment'])
X_test_tfidf = tfidf.transform(X_test['preprocessed_comment'])

# Kết hợp đặc trưng văn bản với các đặc trưng số học
import scipy.sparse
X_train_combined = scipy.sparse.hstack([X_train_tfidf, X_train[['comment_length', 'positive_ratio', 'negative_ratio']].values])
X_test_combined = scipy.sparse.hstack([X_test_tfidf, X_test[['comment_length', 'positive_ratio', 'negative_ratio']].values])

In [ ]:
X_train.head()

## Logistic Regression

- Đơn giản, dễ huấn luyện và giải thích.
- Tốt với các bài toán tuyến tính.
- Có thể là baseline để so sánh với các mô hình khác.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Huấn luyện Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train_combined, y_train)

# Dự đoán
y_pred = lr_model.predict(X_test_combined)

# Đánh giá
print("Logistic Regression Report:")
print(classification_report(y_test, y_pred))


## Random Forest

- Mạnh mẽ với dữ liệu không tuyến tính.
- Không cần chuẩn hóa dữ liệu.
- Có thể tính được feature importance, giúp hiểu rõ đóng góp của từng feature.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Định nghĩa mô hình
rf = RandomForestClassifier(random_state=42)

# Định nghĩa bộ tham số cần tìm
param_grid = {
    'n_estimators': [100, 200, 300],         # Số lượng cây
    'max_depth': [None, 10, 20, 30],        # Độ sâu tối đa của mỗi cây
    'min_samples_split': [2, 5, 10],        # Số lượng mẫu tối thiểu để tách
    'min_samples_leaf': [1, 2, 4],          # Số lượng mẫu tối thiểu ở mỗi lá
    'bootstrap': [True, False]              # Sử dụng bootstrap hay không
}

# Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, scoring='f1_weighted', verbose=2, n_jobs=-1)

# Huấn luyện
grid_search.fit(X_train_combined, y_train)

# Kết quả tối ưu
print("Best Parameters:", grid_search.best_params_)
print("Best F1-Score:", grid_search.best_score_)


## Gradient Boosting (XGBoost)

In [ ]:
from xgboost import XGBClassifier

# Huấn luyện XGBoost
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_combined, y_train)

# Dự đoán
y_pred = xgb_model.predict(X_test_combined)

# Đánh giá
print("XGBoost Report:")
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot()
plt.show()
